# Efficiencies

In [1]:
import sys
import localSettings as ls
import os
print(ls.main_path)

/uboone/app/users/cthorpe/PELEE/


In [2]:
main_path = ls.main_path
sys.path.append(main_path)

In [3]:
from datetime import datetime
now = datetime.now()
date_time = now.strftime("%m%d%Y")
print("date and time:",date_time)

date and time: 06092023


In [4]:
from scipy.optimize import curve_fit

def gauss(x,mu,sigma,A):
    norm = A/(np.sqrt(2*np.pi)*sigma)
    exp  = np.exp(-((x-mu)**2)/(2*sigma*sigma))
    return norm * exp

In [5]:
import plotter
import importlib
importlib.reload(plotter)
import uproot
import matplotlib.pylab as pylab
import numpy as np
import math
from sklearn.model_selection import train_test_split
import pickle
#if USEBDT:
import xgboost as xgb
import nue_booster 
importlib.reload(nue_booster)
import awkward
import pandas as pd

params = {
    'axes.labelsize': 'x-large',
    'axes.titlesize': 'x-large',
    'xtick.labelsize': 'x-large',
    'ytick.labelsize': 'x-large'
}
pylab.rcParams.update(params)

In [21]:
import load_data_run123
VARDICT = load_data_run123.get_variables()

In [22]:
loadnues = True

In [29]:
#PATH = '/home/david/data/searchingfornues/v08_00_00_43/0919/'
PATH = ls.ntuple_path + "run2/nuepresel/"
if (loadnues):
    FILE = 'prodgenie_bnb_intrinsic_nue_overlay_run2_v08_00_00_35_run2a_reco2_reco2.root'
else:
    FILE = 'prodgenie_bnb_nu_uboone_overlay_mcc9.1_v08_00_00_26_filter_run2_reco2_D1D2_reco2.root'

In [30]:
fold = 'nuselection'
tree = 'NeutrinoSelectionFilter'

In [31]:
if (loadnues):
    # nues
    #variables = load_data_run123.VARIABLES + load_data_run123.NUEVARS #+ load_data_run123.RCVRYVARS
    variables = VARDICT['VARIABLES'] + VARDICT['NUEVARS']
else:
    # numus
    #variables = load_data_run123.get_variables()#VARIABLES
    variables = VARDICT['VARIABLES']

In [32]:
print ("trk1_nhits" in variables)

False


In [33]:
up = uproot.open(PATH+FILE)[fold][tree]
df = up.pandas.df(variables, flatten=False)

In [34]:
if (loadnues):
    load_data_run123.process_uproot(up,df)
    #load_data_run123.process_uproot_recoveryvars(up,df)
else:
    load_data_run123.process_uproot_numu(up,df)

In [35]:
import unblinding_far_sideband

In [36]:
df['bnbdata']  = np.zeros_like(df['nslice'])
df['extdata']  = np.zeros_like(df['nslice'])

In [38]:
TRAINVAR = ["tksh_distance","tksh_angle",
            "shr_tkfit_dedx_max",
            "trkpid",
            "trkshrhitdist2","hits_ratio"]
            

LABELS =  ['pi0','nonpi0']

TRAINVARZP = [
              "shrMCSMom","DeltaRMS2h","shrPCA1CMed_5cm","CylFrac2h_1cm"]

In [39]:
TRAINVAR = ["shr_score","tksh_distance","tksh_angle",
            "shr_tkfit_dedx_max",
            "trkfit","trkpid",
            "subcluster","shrmoliereavg",
            "trkshrhitdist2","hits_ratio",
            "secondshower_Y_nhit","secondshower_Y_vtxdist","secondshower_Y_dot","anglediff_Y",
            "CosmicIPAll3D","CosmicDirAll3D"]

LABELS =  ['pi0','nonpi0']

TRAINVARZP = ['shrmoliereavg','shr_score', "trkfit","subcluster",
              "CosmicIPAll3D","CosmicDirAll3D",
              'secondshower_Y_nhit','secondshower_Y_vtxdist','secondshower_Y_dot','anglediff_Y',
              'secondshower_V_nhit','secondshower_V_vtxdist','secondshower_V_dot','anglediff_V',
              'secondshower_U_nhit','secondshower_U_vtxdist','secondshower_U_dot','anglediff_U',
              "shr_tkfit_2cm_dedx_U", "shr_tkfit_2cm_dedx_V", "shr_tkfit_2cm_dedx_Y",
              "shr_tkfit_gap10_dedx_U", "shr_tkfit_gap10_dedx_V", "shr_tkfit_gap10_dedx_Y",
              "shrMCSMom","DeltaRMS2h","shrPCA1CMed_5cm","CylFrac2h_1cm"]

LABELSZP = ['bkg']

def loadBDT(DF):

    for label, bkg_query in zip(LABELS, nue_booster.bkg_queries):
        with open(ls.pickle_path+'booster_%s_0304_extnumi.pickle' % label, 'rb') as booster_file:
            booster = pickle.load(booster_file)
            DF[label+"_score"] = booster.predict(xgb.DMatrix(DF[TRAINVAR]),
                                                 ntree_limit=booster.best_iteration)

    for label, bkg_query in zip(LABELSZP, nue_booster.bkg_queries):
        with open(ls.pickle_path+'booster_%s_0304_extnumi_vx.pickle' % label, 'rb') as booster_file:
            booster = pickle.load(booster_file)
            DF[label+"_score"] = booster.predict(xgb.DMatrix(DF[TRAINVARZP]),
                                                 ntree_limit=booster.best_iteration)
            
    return

In [44]:
INTERCEPT = 0.0
SLOPE = 0.83

if (loadnues):
    df['subcluster'] = df['shrsubclusters0'] + df['shrsubclusters1'] + df['shrsubclusters2']
    df['trkfit'] = df['shr_tkfit_npointsvalid'] / df['shr_tkfit_npoints']
    df['anglediff_Y'] = np.abs(df['secondshower_Y_dir']-df['shrclusdir2'])
    df['anglediff_V'] = np.abs(df['secondshower_V_dir']-df['shrclusdir1'])
    df['anglediff_U'] = np.abs(df['secondshower_U_dir']-df['shrclusdir0'])                                                                                                                       
    df["ptOverP"] = df["pt"]/df["p"]
    df["phi1MinusPhi2"] = df["shr_phi"]-df["trk_phi"]
    df["theta1PlusTheta2"] = df["shr_theta"]+df["trk_theta"]
    df['cos_shr_theta'] = np.cos(df['shr_theta'])
    df['shr_tkfit_nhits_tot'] = (df['shr_tkfit_nhits_Y']+df['shr_tkfit_nhits_U']+df['shr_tkfit_nhits_V'])                                                                                                                                                                                              
    df['shr_tkfit_2cm_nhits_tot'] = (df['shr_tkfit_2cm_nhits_Y']+df['shr_tkfit_2cm_nhits_U']+df['shr_tkfit_2cm_nhits_V'])                                                                                                                                                              
    df['shr_tkfit_gap10_nhits_tot'] = (df['shr_tkfit_gap10_nhits_Y']+df['shr_tkfit_gap10_nhits_U']+df['shr_tkfit_gap10_nhits_V'])                                                                                                                             
    df.loc[:,'shr_tkfit_dedx_max'] = df['shr_tkfit_dedx_Y']
    df.loc[(df['shr_tkfit_nhits_U']>df['shr_tkfit_nhits_Y']),'shr_tkfit_dedx_max'] = df['shr_tkfit_dedx_U']
    df.loc[(df['shr_tkfit_nhits_V']>df['shr_tkfit_nhits_Y']) & (df['shr_tkfit_nhits_V']>df['shr_tkfit_nhits_U']),'shr_tkfit_dedx_max'] = df['shr_tkfit_dedx_V']
    df["reco_e"] = (df["shr_energy_tot_cali"] + INTERCEPT) / SLOPE + df["trk_energy_tot"]
    df['electron_e'] = (df["shr_energy_tot_cali"] + INTERCEPT) / SLOPE

    loadBDT(df)

In [41]:
if (loadnues):
    #QUERY = unblinding_far_sideband.NPPRESQ + ' and ' + unblinding_far_sideband.BDTCQ
    QUERY = unblinding_far_sideband.ZPPRESEL + ' and ' + unblinding_far_sideband.ZPBDTLOOSE
else:
    QUERY = unblinding_far_sideband.NUMUPRESEL + ' and n_muons_tot > 0'

In [42]:
print (QUERY)

nslice == 1 and selected == 1 and shr_energy_tot_cali > 0.07 and ( (_opfilter_pe_beam > 0 and _opfilter_pe_veto < 20) or bnbdata == 1 or extdata == 1) and n_tracks_contained == 0 and nslice == 1 and selected == 1 and shr_energy_tot_cali > 0.07 and ( (_opfilter_pe_beam > 0 and _opfilter_pe_veto < 20) or bnbdata == 1 or extdata == 1) and n_showers_contained == 1 and CosmicIPAll3D > 10. and CosmicDirAll3D > -0.9 and CosmicDirAll3D < 0.9 and shrmoliereavg < 15 and subcluster > 4 and trkfit < 0.65 and secondshower_Y_nhit < 50 and shr_trk_sce_start_y > -100 and shr_trk_sce_start_y < 80 and shr_trk_sce_end_y > -100 and shr_trk_sce_end_y < 100  and shr_trk_len < 300. and (n_tracks_tot == 0 or (n_tracks_tot>0 and tk1sh1_angle_alltk>-0.9)) and bkg_score >0.72 and n_tracks_contained == 0 and (n_tracks_tot == 0 or (n_tracks_tot>0 and tk1sh1_angle_alltk>-0.9))


In [43]:
dfsel = df.query(QUERY)

UndefinedVariableError: name 'tk1sh1_angle_alltk' is not defined

In [ ]:
if (loadnues):
    #fout = open(PATH+'SBNFit/'+"1eNp"+'.txt','w')
    fout = open(PATH+'SBNFit/'+"1e0p"+'.txt','w')
else:
    fout = open(PATH+'SBNFit/'+"numu"+'.txt','w')

print ('sample %i selected entries'%(dfsel.shape[0]))
for i,row in dfsel.iterrows():
    run = row['run']
    sub = row['sub']
    evt = row['evt']
    nu_e = row['nu_e']
    if (loadnues):
        reco_e = row['reco_e']
        fout.write('%i %i %i %f %.02f \n'%(run,sub,evt,nu_e,reco_e))
    else:
        nuenergy = row['neutrino_energy']
        mutheta  = row['muon_theta']
        muenergy = row['muon_energy']
        if (nuenergy > 0):
            fout.write('%i %i %i %f %.02f %.02f %.02f \n'%(run,sub,evt,nu_e,nuenergy,mutheta,muenergy))
fout.close()